<a href="https://colab.research.google.com/github/awadell1/MLOnAGooseberryBush/blob/master/src/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import text_to_word_sequence
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
SEED = 42 #setting the seed for reproducebility
AUTOTUNE = tf.data.experimental.AUTOTUNE #currently not entirely sure what this can do

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
save_path='/content/drive/MyDrive/24787 Group Project  /word2vec model/vectors256.tsv'
word2vec_map = pd.read_csv(save_path,sep='\t',header=None)
print(word2vec_map.shape)
word2vec_map.head(2)

(33662, 256)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,-0.130494,0.073643,-0.354955,-0.075573,-0.061752,-0.399214,-0.140066,0.143229,-0.183003,-0.033217,0.048220,0.030298,-0.052610,0.126930,-0.330439,0.271223,-0.010008,-0.366288,-0.033729,-0.437847,0.010464,0.155762,-0.185933,-0.436063,-0.210882,0.162651,0.277120,0.027856,0.125649,-0.326646,-0.098107,0.058052,0.146102,-0.367666,-0.325120,0.211603,0.088326,0.087743,-0.056309,-0.169389,...,0.082857,-0.184996,0.249066,0.083633,-0.096283,-0.051716,0.122224,0.084957,-0.075601,0.041088,0.118679,-0.217012,0.282111,0.495631,0.498411,-0.239457,0.009129,-0.400434,-0.257665,0.260823,-0.326247,-0.141764,0.099803,0.018076,0.092956,0.235196,0.227048,0.027845,0.029719,-0.069874,-0.280754,0.227582,-0.108462,-0.031027,-0.303575,-0.054791,0.249680,-0.060876,0.314991,-0.192157
1,-0.234385,0.227516,-0.106715,-0.200246,-0.203092,0.007012,0.220154,0.239702,-0.044499,0.262064,0.075031,0.431765,-0.123586,-0.097911,-0.017337,0.041602,0.314082,-0.070995,-0.343952,0.141355,-0.397809,0.046767,0.068449,-0.140294,-0.404553,0.028795,0.252434,-0.152068,0.361540,-0.404193,0.369497,-0.076468,-0.176338,0.044829,-0.052121,0.256241,0.196653,-0.091581,-0.716998,-0.407812,...,0.163593,-0.270718,0.467247,-0.386281,0.200450,-0.191414,0.278099,-0.292865,0.023086,-0.033929,-0.441728,0.140238,0.067459,0.464505,0.062631,-0.159951,0.062956,0.330492,-0.551830,-0.076356,0.182487,0.150197,-0.013580,-0.040434,0.026686,-0.295480,-0.111801,0.097040,-0.113225,-0.175610,0.188318,0.031296,0.205229,-0.091539,-0.103013,-0.212915,0.053797,-0.219028,0.075542,-0.008862


In [ ]:
I2W_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/Dictionary_new.csv')

In [ ]:
I2W=I2W_df.to_dict(orient='dict')['Vocab']
I2W[0] = ''
W2I = {v: k for k, v in I2W.items()}

In [ ]:
#load the data
raw_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
raw_data=raw_df[['description']]
raw_label=raw_df[['points']]
sentences_num=[]
for i, row in tqdm(raw_data.iterrows()):
    tokens = text_to_word_sequence(row[0],filters="—!#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r“”""")
    temp_mem=[W2I[word] for word in tokens]
    sentences_num.append(temp_mem)

In [ ]:
max_sentence_length = max([len(seq) for seq in sentences_num])
input_sequences = np.array(pad_sequences(sentences_num, maxlen=max_sentence_length, padding='pre'))

In [ ]:
for seq in input_sequences[:3]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0    15   868   201    13  1975  3601     1    88
   105     2    16   899  1019   919   306 14837    48    60     1    88
   417   142   359    18] 
 => 
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
I2W_for_embedding = I2W.copy()
I2W_for_embedding.pop(0)
W2I_for_embedding = {v: k for k, v in I2W_for_embedding.items()}

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index)+1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map.shape[1]      # define dimensionality of your word vectors (= 256)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in tqdm(word_to_index.items()): #skip the first item "<padding>"
        emb_matrix[index, :] = word_to_vec_map.iloc[index-1] ###TO-DO:update the word2vec

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim,trainable=True) 
    ##TODO:is this maskz_zero correct here?
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word2vec_map, W2I_for_embedding)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = -0.07557311


In [ ]:
def WineReviewMachine_V2(input_shape, word_to_vec_map, word_to_index,num_class):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(num_class)(X)
    # Add a softmax activation
    X = Activation("softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
num_class=5
model = WineReviewMachine_V2((max_sentence_length,), word2vec_map, W2I_for_embedding, num_class)
model.summary()


Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 136, 256)          8617472   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 136, 512)          1050624   
_________________________________________________________________
dropout_14 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)              

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
y_label=raw_label.values
y_label=y_label-y_label.min()
y_label_5levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <4:
    temp = int(0)
  elif y0>=4 and y0<8:
     temp = int(1.0)
  elif y0>=8 and y0<12:
     temp = int(2.0)
  elif y0>=12 and y0<16:
     temp = int(3.0)
  else: temp = int(4.0)
  y_label_5levels.append(temp)

In [ ]:
indices = y_label_5levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:103976]
test_ind = targ_ind[103976:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 5)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 5)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 5)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 5)), types: (tf.int32, tf.float32)>


In [ ]:
#some model visualization function, copied from Word2Vec tutorial
tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs5") 

#change the learning rate when plateau
#lr = ReduceLROnPlateau(patience=3, verbose=True) 

#early stop the model to prevent overfit (stop when the validation accuracy is not improving much)
#es = EarlyStopping(patience=5, verbose=True, restore_best_weights=True) 

In [ ]:
model.fit(dataset_train,callbacks=tensorboard,epochs=42,verbose=1,validation_data=dataset_test)

Epoch 1/42
101/101 [==============================] - 56s 554ms/step - loss: 0.8894 - accuracy: 0.6019 - val_loss: 0.7248 - val_accuracy: 0.6758
Epoch 2/42
101/101 [==============================] - 55s 542ms/step - loss: 0.7015 - accuracy: 0.6933 - val_loss: 0.6881 - val_accuracy: 0.6928
Epoch 3/42
101/101 [==============================] - 55s 541ms/step - loss: 0.6486 - accuracy: 0.7187 - val_loss: 0.6762 - val_accuracy: 0.6987
Epoch 4/42
101/101 [==============================] - 55s 541ms/step - loss: 0.6076 - accuracy: 0.7401 - val_loss: 0.6879 - val_accuracy: 0.6983
Epoch 5/42
101/101 [==============================] - 55s 541ms/step - loss: 0.5706 - accuracy: 0.7600 - val_loss: 0.7256 - val_accuracy: 0.6932
Epoch 6/42
101/101 [==============================] - 55s 541ms/step - loss: 0.5469 - accuracy: 0.7718 - val_loss: 0.7150 - val_accuracy: 0.6957
Epoch 7/42
101/101 [==============================] - 55s 541ms/step - loss: 0.5218 - accuracy: 0.7839 - val_loss: 0.7438 - val_ac

In [ ]:
model.fit(dataset_train,callbacks=tensorboard,epochs=300,verbose=1,validation_data=dataset_test)

Epoch 1/300
101/101 [==============================] - 88s 870ms/step - loss: 1.0989 - accuracy: 0.4906 - val_loss: 0.9592 - val_accuracy: 0.5647
Epoch 2/300
101/101 [==============================] - 81s 801ms/step - loss: 0.9473 - accuracy: 0.5680 - val_loss: 0.9251 - val_accuracy: 0.5782
Epoch 3/300
101/101 [==============================] - 81s 801ms/step - loss: 0.8980 - accuracy: 0.5912 - val_loss: 0.9084 - val_accuracy: 0.5843
Epoch 4/300
101/101 [==============================] - 81s 801ms/step - loss: 0.8536 - accuracy: 0.6147 - val_loss: 0.8901 - val_accuracy: 0.5910
Epoch 5/300
101/101 [==============================] - 81s 801ms/step - loss: 0.8239 - accuracy: 0.6296 - val_loss: 0.8520 - val_accuracy: 0.6104
Epoch 6/300
101/101 [==============================] - 81s 802ms/step - loss: 0.7942 - accuracy: 0.6445 - val_loss: 0.7987 - val_accuracy: 0.6436
Epoch 7/300
101/101 [==============================] - 81s 801ms/step - loss: 0.7748 - accuracy: 0.6531 - val_loss: 0.7925 -

KeyboardInterrupt: ignored

rmdir: failed to remove 'logs': No such file or directory
